# Load and prepare data
**Objective**: Load news and tweets data from raw data files into sqlite3 db.

# Roadmap
1. Copy ~~Meng~~ original data folder to DATA_DIR, unzip and check format.
2. Create db. Build tables for news and tweets.
3. Bulk load news and tweets data into db.
4. Check basic statistics.
4. Build df pickles for news and tweets over selected period

# Steps

In [1]:
"""
Initialization
"""

'''
Standard modules
'''
import os
import sqlite3
import csv
import time
import codecs
from pprint import pprint

'''
Analysis modules
'''
import pandas as pd

'''
Custom modules
'''
import config
import utilities

'''
Misc
'''
nb_name = '20171002-daheng-load_and_prepare_data'

## Copy ~~Meng~~ original data folder to DATA_DIR, unzip and check format.

In [2]:
"""
Register
    MENG_NEWS_TWEETS_DIR = os.path.join(DATA_DIR, 'raw-news_tweets-meng') DEPRECATED
    ORIGINAL_NEWS_TWEETS_DIR = os.path.join(DATA_DIR, 'raw-news_tweets-original')
in config.
"""

"\nRegister\n    MENG_NEWS_TWEETS_DIR = os.path.join(DATA_DIR, 'raw-news_tweets-meng') DEPRECATED\n    ORIGINAL_NEWS_TWEETS_DIR = os.path.join(DATA_DIR, 'raw-news_tweets-original')\nin config.\n"

Raw data files are located inside ORIGINAL_NEWS_TWEETS_DIR folder.  

In [3]:
! ls -1 ./data/raw-news_tweets-original/

dataset1
dataset2


In [4]:
! ls ./data/raw-news_tweets-original/dataset1/news

2014-11-18.txt	2014-12-28.txt	2015-02-05.txt	2015-03-16.txt	2015-04-30.txt
2014-11-19.txt	2014-12-29.txt	2015-02-06.txt	2015-03-17.txt	2015-05-01.txt
2014-11-20.txt	2014-12-30.txt	2015-02-07.txt	2015-03-18.txt	2015-05-02.txt
2014-11-21.txt	2014-12-31.txt	2015-02-08.txt	2015-03-19.txt	2015-05-03.txt
2014-11-22.txt	2015-01-01.txt	2015-02-09.txt	2015-03-20.txt	2015-05-04.txt
2014-11-23.txt	2015-01-02.txt	2015-02-10.txt	2015-03-21.txt	2015-05-05.txt
2014-11-24.txt	2015-01-03.txt	2015-02-11.txt	2015-03-22.txt	2015-05-06.txt
2014-11-25.txt	2015-01-04.txt	2015-02-12.txt	2015-03-29.txt	2015-05-07.txt
2014-11-26.txt	2015-01-05.txt	2015-02-13.txt	2015-03-30.txt	2015-05-08.txt
2014-11-27.txt	2015-01-06.txt	2015-02-14.txt	2015-03-31.txt	2015-05-09.txt
2014-11-28.txt	2015-01-07.txt	2015-02-15.txt	2015-04-01.txt	2015-05-10.txt
2014-11-29.txt	2015-01-08.txt	2015-02-16.txt	2015-04-02.txt	2015-05-11.txt
2014-11-30.txt	2015-01-09.txt	2015-02-17.txt	2015-04-03.txt	2015-05-12.txt
2014-12-01.txt	2015-01-10

In [5]:
! ls ./data/raw-news_tweets-original/dataset1/tweets/2014-11-18/

2685_Missouris_Nixon_Declares_State_of_Emergency_Awaiting_Grand_Jury-Businessweek
2686_PEOPLE_Bill_Cosby_Charles_Manson_Solange_Knowles_and_more-Pittsburgh_Post_Gazette
2687_Ebola_patient_who_died_had_received_ZMapp_late_in_his_treatment-Los_Angeles_Times
2688_At_least_4_dead_in_attack_in_Kabul_official_says-Fox_News
2689_Australia_will_not_be_at_periphery_of_Indias_vision_Modi-The_Hindu
2690_FBI_Violence_could_follow_Ferguson_indictment_decision-CBS_News
2691_Four_Killed_in_Palestinian_Attack_at_Jerusalem_Synagogue-Businessweek
2692_Mass_murderer_Charles_Manson_issued_marriage_license_may_get_hitched_next_-New_York_Daily_News
2693_News_Guide_Texas_latest_history_textbook_tussle-Washington_Post
2694_AbdulRahman_Kassigs_parents_mourn_beloved_son-BBC_News
2695_Obama_orders_full_review_of_US_hostage_policy-Fox_News
2696_Homeless_Children_in_US_A_parenttoparent_approach_to_help_kids_video-Christian_Science_Monitor
2697_Alleged_Bill_Cosby_victim_has_connection_to_Colorado-KRDO
2698_Church_o

### Check format of news data:

In [6]:
! head -1 ./data/raw-news_tweets-original/dataset1/news/2014-11-18.txt

2685	http://www.businessweek.com/news/2014-11-17/missouri-s-nixon-declares-state-of-emergency-awaiting-grand-jury	Missouri's Nixon Declares State of Emergency Awaiting Grand Jury	Businessweek	Mon, 17 Nov 2014 21:04:21 -0800	Toluse Olorunnipa	missouri;nixon declares;emergency awaiting;grand jury	Missouri Governor Jay Nixon's actions underscore rising tensions in the St. Louis region, which has seen months of protests since the Aug. 9 shooting death of 18-year-old Michael Brown. Photographer: Scott Olson/Getty Images. Governor Jay Nixon declared ...	Missouri Governor Jay Nixon’s actions underscore rising tensions in the St. Louis region, which has seen months of protests since the Aug. 9 shooting death of 18-year-old Michael Brown. Photographer: Scott Olson/Getty Images::::::::Governor Jay Nixon activated the Missouri National Guard in anticipation of unrest when a grand jury decides whether to indict a white police officer who shot and killed an unarmed black teenager.::::::::Nixon decl

### Check format of tweets data:

In [7]:
! head -3 ./data/raw-news_tweets-original/dataset1/tweets/2014-11-18/2685_Missouris_Nixon_Declares_State_of_Emergency_Awaiting_Grand_Jury-Businessweek

534598949748092928	Missouri governor declares emergency, activates National Guard ahead of grand jury announcement... http://t.co/iXqEprM4KJ	Tue Nov 18 06:48:16 +0000 2014	http://t.co/iXqEprM4KJ							False	0	False	0	104591943	False	567	3187	913	None	Wed Jan 13 20:12:34 +0000 2010			Facebook			
534598876976910336	Missouri+governor+declares+state+of+emergency+ahead+of+Ferguson+grand+jury+decision+(+video)+-+http://t.co/XCnLNeibfk http://t.co/9byi9KURzB	Tue Nov 18 06:47:59 +0000 2014	http://t.co/XCnLNeibfk,http://t.co/9byi9KURzB							False	0	False	0	40676472	False	1002	1227	2001	None	Sun May 17 15:27:05 +0000 2009	Orlando, FL		Twitter for Android			
534598442820317184	RT @TeaPartyCat: Missouri governor declares state of racism in Ferguson in anticipation of grand jury deciding that killing a black man isn…	Tue Nov 18 06:46:15 +0000 2014			534570486706216960	False	93	False	113	False	0	False	113	1699988293	False	173	2732	281	None	Sun Aug 25 19:21:04 +0000 2013	Spokane, wa		Twitter for iPh

## Create db. Build tables for news and tweets.

### Define DDL scripts to create db, table schema for news, and table schema for tweets

In [8]:
"""
Register
    NEWS_TWEETS_DDL_FILE = os.path.join(DATA_DIR, 'original-news_tweets.schema.sql')
in config.
"""

"\nRegister\n    NEWS_TWEETS_DDL_FILE = os.path.join(DATA_DIR, 'original-news_tweets.schema.sql')\nin config.\n"

In [9]:
! cat ./data/original-news_tweets.schema.sql

-- original-news_tweets.schema.sql

-- Schema for news and tweets data

-- News table
create table news (
    news_id integer primary key autoincrement not null,
    news_native_id integer not null,
    news_url text,
    news_title text,
    news_source text,
    news_post_time datetime,
    news_collected_time date,
    news_keywords text,
    news_doc text
);

-- Tweets table
create table tweets (
    tweet_id integer primary key autoincrement not null,
    tweet_native_id integer not null,
    tweet_text text,
    tweet_post_time datetime,
    tweet_collected_time date,
    news_native_id integer not null,
    foreign key (news_native_id) references news (news_native_id) on delete cascade
);



### Build db and talbes

In [10]:
"""
Register 
    NEWS_TWEETS_DB = os.path.join(DATA_DIR, 'news_tweets-meng.db')
in config.
"""
def check_db():
    db_exists = os.path.exists(config.NEWS_TWEETS_DB_FILE)
    
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        if not db_exists:
            print('No db exists. Creating new db and tables...')
            with open(config.NEWS_TWEETS_DDL_FILE, 'r') as f:
                schema = f.read()
                conn.executescript(schema)
        else:
            print('db already exists!')

In [11]:
if 1 == 1:
    check_db()

No db exists. Creating new db and tables...


## Bulk load news and tweets into db

### Load news data into db table

In [13]:
%%time
"""
Parse and load original news data

Note: should be executed two times
 - config.ORIGINAL_NEWS_TWEETS_DIR/dataset1/news/
 - config.ORIGINAL_NEWS_TWEETS_DIR/dataset2/news/

"""
if 0 == 1:
    '''
    Get all [date].txt news files
    '''
    news_dir = os.path.join(config.ORIGINAL_NEWS_TWEETS_DIR, 'dataset1', 'news')
    news_files = os.listdir(news_dir)
    # sort files by date
    news_files.sort(key=lambda x: time.strptime(x, '%Y-%m-%d.txt'))

    '''
    Define news file line format information
    '''
    csv.register_dialect('original_news_line', delimiter='\t', doublequote=False, quoting=csv.QUOTE_NONE)
    news_line_index_dict = {'news_native_id': 0, 'news_url': 1, 'news_title': 2, 'news_source': 3, 'news_post_time': 4, 'news_keywords': 6, 'news_doc': 7}
    
    query = '''insert into news (news_native_id, news_url, news_title, news_source, news_post_time, news_collected_time, news_keywords, news_doc) values (?, ?, ?, ?, ?, ?, ?, ?)'''
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        '''
        For each news file, read-in, parse, and load into db news table
        '''
        for news_file in news_files:
            # parse out news collected date from the name of the file
            news_collected_time = os.path.split(news_file)[1].split('.')[0]
            with open(os.path.join(news_dir, news_file), 'r') as f:
                news_tpl_lst = []
                news_lines = csv.reader(f, dialect='original_news_line')
                for news_line in news_lines:
                    news_tpl = (news_line[news_line_index_dict['news_native_id']],
                                news_line[news_line_index_dict['news_url']],
                                news_line[news_line_index_dict['news_title']],
                                news_line[news_line_index_dict['news_source']],
                                utilities.parse_news_post_time(news_line[news_line_index_dict['news_post_time']]),
                                news_collected_time,
                                news_line[news_line_index_dict['news_keywords']],
                                news_line[news_line_index_dict['news_doc']])
                    
                    news_tpl_lst.append(news_tpl)
                
                cursor.executemany(query, news_tpl_lst)

CPU times: user 2.28 s, sys: 88 ms, total: 2.37 s
Wall time: 2.44 s


### Load tweets data into db table

In [17]:
%%time
"""
Parse and load original tweets data
"""
if 0 == 1:
    '''
    Retrieve all valid dates for news
    '''
    news_valid_dates_lst = []
    
    query_news_valid_dates = '''
    select distinct news_collected_time from news
    order by news_collected_time;'''
    
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute(query_news_valid_dates)
        news_valid_dates_lst = [item[0] for item in cursor.fetchall()]
    
    '''
    Tmp var to manipulate with valid date lst in case of unexpected errors when building db
    ''' 
    tmp_lst = []
    for news_valid_date in news_valid_dates_lst:
        if news_valid_date not in config.ORIGINAL_TWEETS_ERROR_DATES_LST:
            tmp_lst.append(news_valid_date)
            
    news_valid_dates_lst = tmp_lst
    
    '''
    Establie conn to db for each news date and commit inserts
    '''
    for news_valid_date in news_valid_dates_lst:
        localtime = time.asctime(time.localtime(time.time()))
        print('Processing tweets associated with news on {} ({})...'.format(news_valid_date, localtime))
        '''
        Retrieve news_native_id from db
        '''
        news_native_ids_lst = []

        query_select_news = '''
        select news_native_id from news
        where news_collected_time = :news_valid_date
        order by news_native_id asc;'''
        with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(query_select_news, {'news_valid_date': news_valid_date})
            news_native_ids_lst = [item[0] for item in cursor.fetchall()]

        '''
        Define tweets file line format information
        '''
        csv.register_dialect('original_tweet_line', delimiter='\t', doublequote=False, quoting=csv.QUOTE_NONE)
        tweet_line_index_dict = {'tweet_native_id': 0, 'tweet_text': 1, 'tweet_post_time': 2}

        query_insert_tweets = '''insert into tweets (tweet_native_id, tweet_text, tweet_post_time, tweet_collected_time, news_native_id) values (?, ?, ?, ?, ?)'''

        with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
            cursor = conn.cursor()
            '''
            For each news:
             - locate corresponding tweets file
             - read-in file lines
             - parse content
             - load into tweets table
            '''
            for news_ind, news_native_id in enumerate(news_native_ids_lst):
                news_collected_time = news_valid_date
                # print out progress info every 100 news articles
#                 if not news_ind % 100:
#                     print('Processing tweets associated with news {} on {} ({}/{})...'.format(news_native_id, news_collected_time, news_ind+1, len(news_native_ids_lst)))
                # determine if the news_tweets file is in dataset1 folder or dataset2 folder
                if news_collected_time <= '2015-06-05':
                    news_tweets_file_dir = os.path.join(config.ORIGINAL_NEWS_TWEETS_DIR, 'dataset1', 'tweets', news_collected_time)
                else:
                    news_tweets_file_dir = os.path.join(config.ORIGINAL_NEWS_TWEETS_DIR, 'dataset2', 'tweets', news_collected_time)

                # find out the name of the news_tweets file
                tweets_files_lst = [file for file in os.listdir(news_tweets_file_dir) if file.startswith(str(news_native_id))]
                # if news_tweets file exists and its size larger than 0
                if tweets_files_lst and os.stat(os.path.join(news_tweets_file_dir, tweets_files_lst[0])).st_size > 0:
                    news_tweets_file = tweets_files_lst[0]
                    with open(os.path.join(news_tweets_file_dir, news_tweets_file), 'r') as f:
                        tweets_tpl_lst = []
                        # get rid of '\0' (Error: line contains NULL byte)
                        tweet_lines = csv.reader((line.replace('\0','') for line in f), dialect='original_tweet_line')
                        for tweet_line in tweet_lines:
                            # filter out non-complete tweet lines and retweets
                            if len(tweet_line) >=3 and not tweet_line[tweet_line_index_dict['tweet_text']].startswith('RT'):
                                # build tpl for tweets table
                                tweet_tpl = (tweet_line[tweet_line_index_dict['tweet_native_id']],
                                             tweet_line[tweet_line_index_dict['tweet_text']],
                                             utilities.parse_tweet_post_time(tweet_line[tweet_line_index_dict['tweet_post_time']]),
                                             news_collected_time,
                                             news_native_id)
                                tweets_tpl_lst.append(tweet_tpl)

                        if tweets_tpl_lst:
                            cursor.executemany(query_insert_tweets, tweets_tpl_lst)

CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 67.1 ms


### Delete news collected on tweet error dates

In [4]:
"""
Twitter internal server was unstable on some dates.
Tweets data collected on these dates contain errors cannot be parsed.
Delete corresponding news data on these dates.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        query_delete_news = '''
        delete from news where news_collected_time = :news_collected_time
        '''
        for original_tweets_error_date in config.ORIGINAL_TWEETS_ERROR_DATES_LST:
            print('Delete news on {} ...'.format(original_tweets_error_date))
            cursor.execute(query_delete_news, {'news_collected_time': original_tweets_error_date})

Delete news on 2015-06-05 ...
Delete news on 2015-09-20 ...
Delete news on 2015-09-21 ...
Delete news on 2015-12-08 ...
Delete news on 2015-12-09 ...
Delete news on 2015-12-10 ...
Delete news on 2016-02-14 ...
Delete news on 2016-02-15 ...
Delete news on 2016-02-17 ...
Delete news on 2016-02-18 ...
Delete news on 2016-02-19 ...


### Build necessary indexes

In [5]:
%%time
"""
Build indexes on news table news_native_id, news_title, news_post_time, and news_collected_time fields.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        queries_lst = ['create index news_news_native_id on news(news_native_id);',
                       'create index news_news_title on news(news_title);',
                       'create index news_news_post_time on news(news_post_time);',
                       'create index news_news_collected_time on news(news_collected_time);']
        
        for query in queries_lst:
            cursor.execute(query)

CPU times: user 236 ms, sys: 132 ms, total: 368 ms
Wall time: 529 ms


In [7]:
%%time
"""
Build indexes on tweets table tweet_native_id, tweet_post_time, tweet_collected_time, and news_native_id fields.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        queries_lst = ['create index tweets_tweet_native_id on tweets(tweet_native_id);',
                       'create index tweets_tweet_post_time on tweets(tweet_post_time);',
                       'create index tweets_tweet_collected_time on tweets(tweet_collected_time);',
                       'create index tweets_news_native_id on tweets(news_native_id);']

        
        for query_ind, query in enumerate(queries_lst):
            print('Building index {}/{} ...'.format(query_ind+1, len(queries_lst)))
            cursor.execute(query)

Building index 1/4 ...
Building index 2/4 ...
Building index 3/4 ...
Building index 4/4 ...
CPU times: user 17min 18s, sys: 6min 45s, total: 24min 3s
Wall time: 50min 45s


In [8]:
"""
List out tables and indices
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        query = """
        select name from sqlite_master
        where type = 'table';
        """
        cursor.execute(query)
        print('TABLES:')
        print(cursor.fetchall())
        
        query = """
        select name from sqlite_master
        where type = 'index'
        """
        cursor.execute(query)
        print('INDICES:')
        print(cursor.fetchall())

TABLES:
[('news',), ('sqlite_sequence',), ('tweets',)]
INDICES:
[('news_news_native_id',), ('news_news_title',), ('news_news_post_time',), ('news_news_collected_time',), ('tweets_tweet_native_id',), ('tweets_tweet_post_time',), ('tweets_tweet_collected_time',), ('tweets_news_native_id',)]


### DEPRECATED CODES

In [ ]:
%%time
"""
DEPRECATED

Load news data by Meng

Note: should be executed two times
 - data/raw-news_tweets-meng/dataset1/news.txt
 - data/raw-news_tweets-meng/dataset2/news.txt
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        with open(os.path.join(config.DATA_DIR, 'raw-news_tweets-meng/dataset2/news.txt'), 'r') as f:
            news_df_chunksize = 10000
            for df_chunk in pd.read_csv(f, sep='\t', chunksize=news_df_chunksize, iterator=True):
                column_names = {
                    'POST_TIME': 'post_time',
                    'NEWS_SOURCE': 'news_source',
                    'NEWS_TITLE': 'news_title',
                    'NEWS_DOC': 'news_doc'
                }
                df_chunk = df_chunk.rename(columns=column_names)
                df_chunk.to_sql(name='news', con=conn, if_exists='append', index=False)

In [ ]:
%%time
"""
DEPRECATED

Load tweets data by Meng

Note: should be executed two times
 - data/raw-news_tweets-meng/dataset1/tweets.txt
 - data/raw-news_tweets-meng/dataset2/tweets.txt
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        with open(os.path.join(config.DATA_DIR, 'raw-news_tweets-meng/dataset2/tweets.txt'), 'r') as f:
            tweets_df_chunksize = 100000
            for df_chunk in pd.read_csv(f, sep='\t', chunksize=tweets_df_chunksize, iterator=True):
                column_names = {
                    'POST_TIME': 'post_time',
                    'TWEET_TEXT': 'tweet_text'
                }
                df_chunk = df_chunk.rename(columns=column_names)
                df_chunk.to_sql(name='tweets', con=conn, if_exists='append', index=False)

## Check basic statistics

In [9]:
%%time
"""
Check number of news per day
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = '''
        select date(news_collected_time) as news_date, count(news_id) as news_num
        from news
        group by date(news_collected_time);
        '''
        
        cursor.execute(query)
        for row in cursor.fetchall():
            print('{}: {}'.format(row['news_date'], row['news_num']))

2014-11-18: 290
2014-11-19: 267
2014-11-20: 320
2014-11-21: 293
2014-11-22: 260
2014-11-23: 275
2014-11-24: 274
2014-11-25: 323
2014-11-26: 307
2014-11-27: 287
2014-11-28: 291
2014-11-29: 250
2014-11-30: 263
2014-12-01: 323
2014-12-02: 291
2014-12-03: 293
2014-12-04: 283
2014-12-05: 288
2014-12-06: 233
2014-12-07: 262
2014-12-08: 292
2014-12-09: 272
2014-12-10: 284
2014-12-11: 284
2014-12-12: 254
2014-12-13: 231
2014-12-14: 149
2014-12-16: 47
2014-12-17: 313
2014-12-18: 51
2014-12-19: 301
2014-12-20: 248
2014-12-21: 268
2014-12-22: 293
2014-12-23: 282
2014-12-24: 287
2014-12-25: 261
2014-12-26: 269
2014-12-27: 253
2014-12-28: 265
2014-12-29: 263
2014-12-30: 69
2014-12-31: 55
2015-01-01: 271
2015-01-02: 282
2015-01-03: 254
2015-01-04: 243
2015-01-05: 262
2015-01-06: 311
2015-01-07: 269
2015-01-08: 275
2015-01-09: 296
2015-01-10: 257
2015-01-11: 282
2015-01-12: 276
2015-01-13: 299
2015-01-14: 311
2015-01-15: 280
2015-01-16: 303
2015-01-17: 248
2015-01-18: 258
2015-01-19: 295
2015-01-20: 

In [11]:
"""
Check any single news article
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = """select * from news where news_native_id = '37293';"""
        
        cursor.execute(query)
        for row in cursor.fetchone():
            print(row)

34609
37293
http://www.foxnews.com/politics/2015/04/03/capitol-hill-reaction-to-framework-for-iran-nuclear-deal-mixed-corker-menendez/
Republicans uneasy over Iran nuke 'deal,' lawmakers demand say on any final agreement
Fox News
2015-04-02 21:17:51
2015-04-03
nuke;Iran;Republicans
The highly touted "framework" for an Iranian nuclear deal, announced Thursday following days of intense negotiations, is being met with mixed reviews on Capitol Hill -- as Republicans voice skepticism and lawmakers on both sides of the aisle reprise ...


In [13]:
%%time
"""
Check number of tweets on a given date
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        tweet_collected_time = '2015-04-03'
        
        query = """
        select tweet_collected_time, count(tweet_id) as tweets_num
        from tweets
        where tweet_collected_time = :tweet_collected_time;
        """
        
        cursor.execute(query, {'tweet_collected_time': tweet_collected_time})
        for row in cursor.fetchall():
            print('{}: {}'.format(row['tweet_collected_time'], row['tweets_num']))

2015-04-03: 1487312
CPU times: user 224 ms, sys: 80 ms, total: 304 ms
Wall time: 602 ms


In [14]:
%%time
"""
Check number of tweets per day
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = """
        select tweet_collected_time, count(tweet_id) as tweets_num
        from tweets
        group by tweet_collected_time;
        """
        
        cursor.execute(query)
        for row in cursor.fetchall():
            print('{}: {}'.format(row['tweet_collected_time'], row['tweets_num']))

2014-11-18: 1494515
2014-11-19: 1147495
2014-11-20: 1439572
2014-11-21: 1320177
2014-11-22: 1305833
2014-11-23: 1354981
2014-11-24: 1246511
2014-11-25: 1244563
2014-11-26: 1593780
2014-11-27: 1227528
2014-11-28: 1546461
2014-11-29: 1236868
2014-11-30: 1393424
2014-12-01: 1457180
2014-12-02: 1515236
2014-12-03: 1336795
2014-12-04: 1265828
2014-12-05: 1308233
2014-12-06: 1367410
2014-12-07: 1272486
2014-12-08: 1214623
2014-12-09: 1346906
2014-12-10: 1188458
2014-12-11: 1273633
2014-12-12: 1210513
2014-12-13: 1029532
2014-12-14: 213366
2014-12-16: 234871
2014-12-17: 924315
2014-12-18: 387406
2014-12-19: 1675156
2014-12-20: 1329957
2014-12-21: 1327250
2014-12-22: 1407439
2014-12-23: 1209476
2014-12-24: 1594981
2014-12-25: 1283703
2014-12-26: 1345370
2014-12-27: 1289327
2014-12-28: 1312207
2014-12-29: 810524
2014-12-30: 55453
2014-12-31: 647316
2015-01-01: 1470963
2015-01-02: 1218878
2015-01-03: 1419915
2015-01-04: 1381914
2015-01-05: 1191842
2015-01-06: 1518042
2015-01-07: 1136823
2015-01-

In [15]:
%%time
"""
Check number of tweets per news
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = """
        select news_native_id, count(tweet_id) as tweets_num
        from tweets
        where tweet_collected_time < '2014-11-20'
        group by news_native_id;
        """
        
        cursor.execute(query)
        for row in cursor.fetchall():
            print('{}: {}'.format(row['news_native_id'], row['tweets_num']))

2685: 1265
2686: 23
2687: 19945
2688: 2036
2689: 15087
2690: 5591
2691: 18219
2692: 5511
2693: 417
2694: 123
2695: 3220
2696: 315
2697: 581
2698: 749
2699: 1202
2700: 519
2701: 10257
2702: 1833
2703: 4656
2704: 685
2705: 106
2706: 180
2707: 238
2708: 39023
2709: 24822
2710: 484
2711: 1596
2712: 130
2713: 42760
2714: 45109
2715: 216
2716: 314
2717: 6584
2718: 4216
2719: 90
2720: 35864
2721: 583
2722: 148
2723: 474
2724: 2313
2725: 182
2726: 63740
2727: 2915
2728: 1263
2729: 285
2730: 24840
2731: 9881
2732: 3592
2733: 18
2734: 154
2735: 235
2736: 324
2737: 18374
2738: 3898
2739: 4998
2740: 1108
2741: 18195
2742: 17848
2743: 341
2744: 57
2745: 110
2746: 349
2747: 52
2748: 255
2749: 7384
2750: 3412
2751: 6089
2752: 130
2753: 103
2754: 26
2755: 20428
2756: 2403
2757: 842
2758: 3529
2759: 106
2760: 3153
2761: 150
2762: 20762
2763: 154
2764: 591
2765: 2310
2766: 103
2767: 2202
2768: 1252
2769: 606
2770: 1672
2772: 20854
2773: 767
2774: 2152
2775: 4587
2776: 10063
2777: 1095
2778: 537
2779: 19

In [19]:
%%time
"""
Check tweets for a given news
"""
if 1 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        
        query = """
        select tweet_native_id, tweet_text, tweet_post_time, tweet_collected_time from tweets
        where tweets.news_native_id = :news_native_id;
        """
        
        cursor.execute(query, {'news_native_id': 3098})
        for row in cursor.fetchall():
            print(row)

(535147631333634049, "Today's six-pack: The sexiest man and the longest bus ride http://t.co/VsNXpbFmsY", '2014-11-19 19:08:32', '2014-11-19')
(535102061751201792, "Today's six-pack: The sexiest man and the longest bus ride http://t.co/GdgzcI05jl", '2014-11-19 16:07:27', '2014-11-19')
(535099134542622720, "Today's six-pack: The sexiest man and the longest bus ride - Bloomington Pantagraph #sixpack #fitness http://t.co/rjCp7dQxtG", '2014-11-19 15:55:49', '2014-11-19')
(535093918036025345, "Today's six-pack: The sexiest man and the longest bus ride http://t.co/HEsWy4KDGd", '2014-11-19 15:35:06', '2014-11-19')
(535092632356020224, "Today's six-pack: The sexiest man and the longest bus ride - http://t.co/SiBp21eBlj: STLtoday.comToday's six-pa... http://t.co/gV16CKE50W", '2014-11-19 15:29:59', '2014-11-19')
(535090575868166144, "#Today's six-pack: The sexiest man and the longest bus ride - http://t.co/Prj4UmbSKx http://t.co/AckfiBlEua", '2014-11-19 15:21:49', '2014-11-19')
(5350883840964976

## Build df pickles for news and tweets over selected period  
Select out news and tweets data from (3) dataset1: 2015-01-01 to 2015-03-21 according to readme.txt file of raw dataset

In [2]:
%%time
"""
Build pickle for news data over selected period.

Register
    NEWS_PERIOD_DF_PKL = os.path.join(DATA_DIR, 'news-period.df.pkl')
in config.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        query = """
        select news_id, post_time, news_source, news_title, news_doc from news
        where post_time between '2015-01-01' and '2015-03-22';
        """
        
        news_period_df = pd.read_sql_query(sql=query,con=conn,parse_dates=['post_time'])
        
        news_period_df.to_pickle(config.NEWS_PERIOD_DF_PKL)

CPU times: user 568 ms, sys: 560 ms, total: 1.13 s
Wall time: 4.22 s


In [3]:
%%time
"""
Build pickle for tweets data over selected period.

Register
    TWEETS_PERIOD_DF_PKL = os.path.join(DATA_DIR, 'tweets-period.df.pkl')
in config.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        query = """
        select tweet_id, post_time, tweet_text from tweets
        where post_time between '2015-01-01' and '2015-03-22';
        """
        
        tweets_period_df = pd.read_sql_query(sql=query,con=conn,parse_dates=['post_time'])
        
        tweets_period_df.to_pickle(config.TWEETS_PERIOD_DF_PKL)

CPU times: user 2min 29s, sys: 26.4 s, total: 2min 56s
Wall time: 3min 29s


In [3]:
%%time
"""
Test recover df pickle
"""
if 1 == 1:
    news_period_df = pd.read_pickle(config.NEWS_PERIOD_DF_PKL)
    tweets_period_df = pd.read_pickle(config.TWEETS_PERIOD_DF_PKL)

CPU times: user 12.4 s, sys: 6.07 s, total: 18.5 s
Wall time: 31.7 s


In [4]:
news_period_df

,news_id,post_time,news_source,news_title,news_doc
0,10965,2015-01-01 00:03:09,CBS News,"Jeb Bush takes ""natural next step"" toward 2016...",Former Florida governor Jeb Bush (R-FL) addres...
1,10966,2015-01-01 00:03:26,CBC.ca,"Fireworks, parties and prayers usher in 2015",Revellers converged at Nathan Phillips Square ...
2,10967,2015-01-01 00:04:41,ABC News,2 Killed in Helicopter Crash in Southern Arizona,Two people were killed in a helicopter crash W...
3,10968,2015-01-01 00:04:41,The Guardian,Jeb Bush quits board posts ahead of possible W...,Potential Republican presidential candidate Je...
4,10969,2015-01-01 00:06:16,NOLA.com,North Korea's Kim Jong Un to South Korean lead...,"PYONGYANG, North Korea -- North Korean leader ..."
5,10970,2015-01-01 00:08:39,Philly.com,Western states get brutal blast of winter,Ryan Winn bundles up son Ian as they wait for ...
6,10971,2015-01-01 00:09:14,Ynetnews,Abbas paves way to join International Criminal...,Following the Palestinian defeat at the UN Sec...
7,10972,2015-01-01 00:12:44,Washington Post,A look at Egypt's Al-Jazeera English trial,CAIRO — Here is a look at the major events in ...
8,10973,2015-01-01 00:16:27,ABC News,"Storm Brings Snow, Cold to West for New Year's",Bundled-up revelers planning to usher in 2015 ...
9,10974,2015-01-01 00:17:17,MiamiHerald.com,"Storm brings snow, cold to West for New Year's",Bundled-up revelers planning to usher in 2015 ...


In [5]:
news_period_df.dtypes

news_id                 int64
post_time      datetime64[ns]
news_source            object
news_title             object
news_doc               object
dtype: object

In [6]:
tweets_period_df

,tweet_id,post_time,tweet_text
0,14225824,2015-01-01 00:03:51,Stampede kills 35 and injures over 40 in New Y...
1,14225825,2015-01-01 00:04:57,good riddance Doug Marrone is no longer @buffa...
2,14225826,2015-01-01 00:04:59,Any real NFL team should aim higher than Kyle ...
3,14225827,2015-01-01 00:05:03,"“@Dan_Lyons76: Man, Doug Marrone has a spectac..."
4,14225828,2015-01-01 00:05:04,How is Doug Marrone better than Rex Ryan?
5,14225829,2015-01-01 00:05:04,Breaking: Buffalo Bills coach Doug Marrone opt...
6,14225830,2015-01-01 00:05:07,Fuck Doug Marrone. I hope he takes Hackett wit...
7,14225831,2015-01-01 00:05:09,You get rid of Rex Ryan for Doug Marrone?!? Lm...
8,14225832,2015-01-01 00:05:10,"Bills: Owner Terry Pegula says he's ""disappoin..."
9,14225833,2015-01-01 00:05:10,Jets: Team will have strong interest in ex-Bil...


In [7]:
tweets_period_df.dtypes

tweet_id               int64
post_time     datetime64[ns]
tweet_text            object
dtype: object